In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
import utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
analysis_date = dt.date(2022, 1, 12)

In [3]:
samtrans_itp_id = 290

In [4]:
importlib.reload(utils)
importlib.reload(rt)

<module 'rt_analysis' from '/home/jovyan/data-analyses/rt_delay/rt_analysis.py'>

In [5]:
colorscale = branca.colormap.step.RdYlGn_10.scale(vmin=0, 
 vmax=30)
colorscale.caption = "Speed (miles per hour)"

In [6]:
from tqdm import tqdm_notebook

In [7]:
from tqdm.notebook import trange, tqdm

In [8]:
pbar = tqdm()

0it [00:00, ?it/s]

In [9]:
## running about 2 minutes for interpolators, about 20 for delay(!)

In [10]:
%%capture
sam = rt.OperatorDayAnalysis(samtrans_itp_id, analysis_date, pbar)

In [13]:
(sam.stop_delay_view
 >> group_by(_.stop_id, _.stop_name)
 >> summarize(mean_delay = _.delay.mean(), max_delay = _.delay.max())
 >> arrange(-_.mean_delay)) >> head(3)

,stop_id,stop_name,mean_delay,max_delay
715,334620,South SF BART-Bay 5 Westside Busway,0 days 00:26:28.198510,0 days 00:26:28.198510
1192,343118,Alameda de las Pulgas & Alma St,0 days 00:18:37.985408200,0 days 00:22:35.850843
269,331100,SF Transit Center - BART shuttle connection,0 days 00:17:02.684474,0 days 00:17:02.684474


In [14]:
sam.set_filter(start_time='15:00', end_time='19:00', direction_id='1')

In [15]:
sam.filter

{'start_time': datetime.time(15, 0),
 'end_time': datetime.time(19, 0),
 'route_ids': None,
 'direction_id': '1',
 'direction': None}

In [16]:
%%capture
m = sam.segment_speed_map(colorscale=colorscale)

In [18]:
# m

### speedup ideas

* selective use of numba @jit decorator in computation-heavy methods?
    * likely requires refactor to plain np, translation, etc...
    * done for time_at_position with wonderful results (60x speedup) 

In [ ]:
# rt.TripPositionInterpolator.detailed_speed_map(ex1)